In [15]:
import os
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_dir = '../DLD'
img_type = 'png'

x = [] # 画像データを格納するリスト
y = [] # ラベルデータを格納するリスト

# img_dir内の全フォルダに対して処理
for folder_name in os.listdir(img_dir):
    folder_path = os.path.join(img_dir, folder_name)
    if os.path.isdir(folder_path):
        # 各フォルダ内の画像ファイルに対して処理
        for file_name in os.listdir(folder_path):
            if file_name.endswith(img_type):
                file_path = os.path.join(folder_path, file_name)
                # 画像を読み込み、サイズを変更
                img = load_img(file_path, target_size=(32, 32))
                # 画像をnumpy配列に変換
                img_array = img_to_array(img)
                # リストに追加
                x.append(img_array)
                # 'DLD'ではなく、実際のフォルダ名をリストに追加
                y.append(folder_name)

# numpy配列に変換
x = np.array(x)
y = np.array(y)



In [16]:
print(x.shape)
print(y.shape)

print(x)
print(y)

(4719, 32, 32, 3)
(4719,)
[[[[169. 169. 169.]
   [170. 170. 170.]
   [170. 170. 170.]
   ...
   [140. 140. 140.]
   [172. 172. 172.]
   [194. 194. 194.]]

  [[180. 180. 180.]
   [183. 183. 183.]
   [187. 187. 187.]
   ...
   [185. 185. 185.]
   [202. 202. 202.]
   [212. 212. 212.]]

  [[182. 182. 182.]
   [190. 190. 190.]
   [200. 200. 200.]
   ...
   [208. 208. 208.]
   [209. 209. 209.]
   [210. 210. 210.]]

  ...

  [[220. 220. 220.]
   [213. 213. 213.]
   [207. 207. 207.]
   ...
   [173. 173. 173.]
   [157. 157. 157.]
   [145. 145. 145.]]

  [[218. 218. 218.]
   [212. 212. 212.]
   [208. 208. 208.]
   ...
   [172. 172. 172.]
   [162. 162. 162.]
   [151. 151. 151.]]

  [[216. 216. 216.]
   [214. 214. 214.]
   [214. 214. 214.]
   ...
   [179. 179. 179.]
   [172. 172. 172.]
   [165. 165. 165.]]]


 [[[193. 193. 193.]
   [166. 166. 166.]
   [131. 131. 131.]
   ...
   [135. 135. 135.]
   [ 90.  90.  90.]
   [ 58.  58.  58.]]

  [[208. 208. 208.]
   [186. 186. 186.]
   [162. 162. 162.]
  

In [17]:
# ラベルデータyの文字列を数値に変換する辞書を作成
label_to_int = {
    '正常': 1,
    '浸潤影': 2,
    'GGO': 3,
    '肺気腫': 4,
    '粒状影': 5,
    '蜂巣肺': 6
}

# yの各要素を数値に変換
y = np.array([label_to_int[label] for label in y])

print(y)


[3 3 3 ... 6 6 6]


In [18]:
from sklearn.model_selection import train_test_split

# xとyを学習用とテスト用に分割する（割合は8:2）
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

x_train.shape: (3775, 32, 32, 3)
x_test.shape: (944, 32, 32, 3)
y_train.shape: (3775,)
y_test.shape: (944,)


In [20]:
from keras.utils import to_categorical

# 特徴量の正規化
x_train = x_train / 255
x_test = x_test / 255

# クラスベクトルの1-hotベクトル化
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [21]:
from keras.models import Sequential

model = Sequential()

In [22]:
from keras.layers import Conv2D

model.add(
    Conv2D(
        filters = 32,
        input_shape = (32, 32, 3),
        kernel_size = (3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu'
    )
)

model.add(
    Conv2D(
        filters = 32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu'
    )
)

c:\Users\Hitoki\anaconda3\envs\tensorflow311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [23]:
from keras.layers import MaxPooling2D

model.add(MaxPooling2D(pool_size=(2, 2)))

In [24]:
from keras.layers import Dropout

model.add(Dropout(0.25))

In [25]:
model.add(
    Conv2D(
        filters = 64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding = 'same',
        activation='relu'
    )
)

model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        activation='relu'
    )
)

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [26]:
model.output_shape

(None, 8, 8, 64)

In [27]:
from keras.layers import Flatten

model.add(Flatten())
model.output_shape

(None, 4096)

In [28]:
from keras.layers import Dense

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

In [29]:
# 作成したモデルのデータへの当てはめ
from keras.callbacks import TensorBoard
import os

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# log_dir = './logs'
# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)

# tsb = TensorBoard(log_dir = log_dir)
history_model1 = model.fit(
    x_train,
    y_train,
    batch_size = 32,
    epochs=20,
    validation_split = 0.2,
    # callbacks = [tsb]
)

Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.4589 - loss: 1.5575 - val_accuracy: 0.5510 - val_loss: 1.0371
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.5516 - loss: 1.0307 - val_accuracy: 0.5497 - val_loss: 1.0470
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.5952 - loss: 0.9262 - val_accuracy: 0.7060 - val_loss: 0.7527
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.6993 - loss: 0.7311 - val_accuracy: 0.7139 - val_loss: 0.7113
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.7622 - loss: 0.6256 - val_accuracy: 0.7788 - val_loss: 0.5192
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.8312 - loss: 0.4591 - val_accuracy: 0.8278 - val_loss: 0.4674
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.8464 - loss: 0.4446 - val_accuracy: 0.8291 - val_loss: 0.4736
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.8418 - loss: 0.4069 - val_accuracy: 0.8066 - v